<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/sixth_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Six Test - Simulate Inference

Inference with the subtask code implementation, the text is now the most reasonable but it has some sense. The only issue is that the COMPLEMENTARY DATA is on blank. But the task is filling it nice. So there exist probability that another probability that we are calling the wrong COMPLEMENTARY DATA.

In [ ]:
import os

# Set this BEFORE importing pytorch/tensorflow
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
print(torch.cuda.device_count()) 

1


In [ ]:
DS_ID = "NONHUMAN-RESEARCH/general-task-index"
PRETRAINED_PATH = "lerobot/pi05_base"

In [ ]:
from xhuman.policies.pi05ki.configuration_pi05ki import PI05KIConfig

policy_config = PI05KIConfig(repo_id="none",device="cuda",pretrained_path=PRETRAINED_PATH)

In [ ]:
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id=DS_ID,
)

In [ ]:
from xhuman.configs.train import TrainPipelineConfigXHUMAN

cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config,
)
cfg.validate()

In [ ]:
from xhuman.datasets.factory import make_dataset_xhuman

cfg.dataset.train_with_subtasks = True
dataset = make_dataset_xhuman(cfg)


In [ ]:
from xhuman.policies.factory import make_xhuman_policy

policy = make_xhuman_policy(
    cfg=cfg.policy,
    ds_meta=dataset.meta,
)

In [ ]:
from xhuman.policies.factory import make_xhuman_pre_post_processors

preprocessor, _ = make_xhuman_pre_post_processors(
        policy_cfg=policy_config,
    )

In [ ]:
device = torch.device("cuda")

In [ ]:
frame = dataset[0]
frame.keys()

dict_keys(['observation.images.left', 'observation.images.top', 'observation.images.right', 'action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index', 'general_task_index', 'action_is_pad', 'task', 'general_task', 'train_with_subtask'])

In [ ]:
from torchvision import transforms
from PIL import Image

# 1. Load the image
img_path_left = 'left.png'
new_left = Image.open(img_path_left).convert("RGB")

# 2. Define the transformation
transform = transforms.Compose([
    transforms.ToTensor()
])

# 3. Convert to tensor
new_left_tensor = transform(new_left)

img_path_right = 'right.png'
new_right = Image.open(img_path_right).convert("RGB")

# 3. Convert to tensor
new_right_tensor = transform(new_right)

img_path_top = 'top.png'
new_top = Image.open(img_path_top).convert("RGB")

# 3. Convert to tensor
new_top_tensor = transform(new_top)

In [ ]:
frame["observation.images.top"].shape

torch.Size([3, 376, 672])

In [ ]:
frame["observation.images.left"] = new_left_tensor
frame["observation.images.right"] = new_right_tensor
frame["observation.images.top"] = new_top_tensor

In [ ]:
# Overwrite the language tokens
# frame["general_task"] = "Task: Move the fruits to the basket. Subtask: "

In [ ]:
batch_subtask = preprocessor.subtask(frame)

In [ ]:
batch_subtask.keys()

dict_keys(['action', 'next.reward', 'next.done', 'next.truncated', 'info', 'action_is_pad', 'task', 'index', 'task_index', 'episode_index', 'train_with_subtask', 'subtask', 'subtask_tokens', 'observation.images.left', 'observation.images.top', 'observation.images.right', 'observation.state', 'observation.language.tokens', 'observation.language.attention_mask'])

In [ ]:
batch_subtask["task"]

['Task: pick the fruits from the table and place them in the basket. Subtask: ']

In [ ]:
images, img_masks = policy._preprocess_images(batch_subtask)

In [ ]:
tokens , masks = batch_subtask["observation.language.tokens"], batch_subtask["observation.language.attention_mask"]

In [ ]:
from xhuman.policies.pi05ki.modeling_pi05ki import make_att_2d_masks
from torch.nn import functional as F

In [ ]:
max_decoding_steps = 200
eos_token_id = 1
temperature = 0.1

In [ ]:
        prefix_embs, prefix_pad_masks, prefix_att_masks = policy.model.embed_prefix(
            images, img_masks, tokens, masks
        )

        prefix_att_2d_masks = make_att_2d_masks(
            prefix_pad_masks, prefix_att_masks
        )

In [ ]:
def _right_align_prefix(embs, pad_masks, att_2d_masks):
            bsz, seq_len = pad_masks.shape
            idx = torch.arange(seq_len, device=pad_masks.device)
            idx = idx[None, :].expand(bsz, seq_len)
            seqlen = (pad_masks.to(idx.dtype) * idx).amax(dim=1) + 1
            gather_idx = (idx + seqlen[:, None]) % seq_len
            gather_idx = gather_idx.long()

            if embs is not None:
                embs = embs.gather(
                    1, gather_idx[:, :, None].expand(-1, -1, embs.shape[-1])
                )
            pad_masks = pad_masks.gather(1, gather_idx)
            att_2d_masks = att_2d_masks.gather(
                1, gather_idx[:, :, None].expand(-1, -1, seq_len)
            )
            att_2d_masks = att_2d_masks.gather(
                2, gather_idx[:, None, :].expand(-1, seq_len, -1)
            )
            return embs, pad_masks, att_2d_masks

        


In [ ]:
prefix_embs, prefix_pad_masks, prefix_att_2d_masks = _right_align_prefix(
            prefix_embs, prefix_pad_masks, prefix_att_2d_masks
        )

In [ ]:
prefix_pad_masks

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, F

In [ ]:
prefix_position_ids = torch.cumsum(prefix_pad_masks, dim=1) - 1

In [ ]:
        # Convert 2D attention masks to 4D format expected by the model
        prefix_att_2d_masks_4d = policy.model._prepare_attention_masks_4d(
            prefix_att_2d_masks, dtype=prefix_embs.dtype
        )

        lang_model = policy.model.paligemma_with_expert.paligemma.language_model
        lang_model.config._attn_implementation = "eager"  # noqa: SLF001

        embeddings, past_key_values = policy.model.paligemma_with_expert.forward(
            attention_mask=prefix_att_2d_masks_4d,
            position_ids=prefix_position_ids,
            past_key_values=None,
            inputs_embeds=[prefix_embs, None],
            use_cache=True,
        )

        # embeddings[0] shape: (B, total_seq_len, embd_dim)
        # Extract last token: (B, embd_dim)
        last_token_embed = embeddings[0][:, -1, :]

        # Convert to logits: (B, vocab_size)
        last_logits = (
            policy.model.paligemma_with_expert.paligemma.lm_head(last_token_embed)
        )

        batch_size = last_logits.shape[0]
        device = last_logits.device

        # prefix_valid_length is the number of valid (non-padded) tokens
        prefix_valid_length = torch.sum(prefix_pad_masks, dim=1)  # (B,)
        output_tokens = torch.zeros((batch_size, max_decoding_steps),
                                    dtype=torch.long, device=device)
        all_eos = torch.zeros(batch_size, dtype=torch.bool, device=device)

        # Prefix attention mask plus the new token attention mask
        running_attention_mask = prefix_pad_masks.clone()

In [ ]:
        # Autoregressive Loop
        for step in range(max_decoding_steps):
            # Sample next token
            if temperature > 0.0:
                probs = F.softmax(last_logits / temperature, dim=-1)
                # token shape: (B, 1)
                token = torch.multinomial(probs, num_samples=1)
            else:
                token = torch.argmax(last_logits, dim=-1, keepdim=True)

            output_tokens[:, step] = token.squeeze(-1)

            # Check for EOS
            all_eos |= (token.squeeze(-1) == eos_token_id)
            if all_eos.all():
                break

            # Feed the new token back in the model
            # token shape: (B, 1) -> embed_tokens returns (B, 1, embd_dim)
            next_token_embeds = lang_model.embed_tokens(token)

            # Create position_ids for the new token
            position_ids = prefix_valid_length[:, None] + step

            # Create attention mask for the new token
            new_mask = torch.ones(
                (batch_size, 1),
                dtype=running_attention_mask.dtype,
                device=device
            )

            running_attention_mask = torch.cat(
                [running_attention_mask, new_mask], dim=1)

            # The attention mask can be 2d or 4d.
            embeds_list, past_key_values = policy.model.paligemma_with_expert.forward(
                inputs_embeds=[next_token_embeds, None],
                attention_mask=running_attention_mask,
                position_ids=position_ids,
                past_key_values=past_key_values,
                use_cache=True,
            )

            print(running_attention_mask)
            prefix_output = embeds_list[0]
            last_token_embed = prefix_output[:, -1, :]
            last_logits = (policy.model.paligemma_with_expert.paligemma.lm_head(last_token_embed))

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, F

In [ ]:
output_tokens

tensor([[3531, 7605,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [ ]:
policy._detokenize_subtask(output_tokens)

'place drop'

In [ ]:
policy._detokenize_subtask(tokens)

'Task: pick the fruits from the table and place them in the basket. Subtask:'